<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/ugcqz6ohbvff804xp84y4kqnvvk3bq1g.png" width = 300, align = "center"></a>

<h1 align=center><font size = 5>Lab: Working with a real world data-set using SQL and Python</font></h1>

# Introduction

This notebook shows how to work with a real world dataset using SQL and Python. In this lab you will:
1. Understand the dataset for Chicago Public School level performance 
1. Store the dataset in an Db2 database on IBM Cloud instance
1. Retrieve metadata about tables and columns and query data from mixed case columns
1. Solve example problems to practice your SQL skills including using built-in database functions

## Chicago Public Schools - Progress Report Cards (2011-2012) 

The city of Chicago released a dataset showing all school level performance data used to create School Report Cards for the 2011-2012 school year. The dataset is available from the Chicago Data Portal: https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t

This dataset includes a large number of metrics. Start by familiarizing yourself with the types of metrics in the database: https://data.cityofchicago.org/api/assets/AAD41A13-BE8A-4E67-B1F5-86E711E09D5F?download=true

Now download a static copy of this database and review some of its contents:
https://ibm.box.com/shared/static/0g7kbanvn5l2gt2qu38ukooatnjqyuys.csv


### Store the dataset in a Table
In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. To analyze the data using SQL, it first needs to be stored in the database.

While it is easier to read the dataset into a Pandas dataframe and then PERSIST it into the database as we saw in the previous lab, it results in mapping to default datatypes which may not be optimal for SQL querying. For example a long textual field may map to a CLOB instead of a VARCHAR. 

Therefore, __it is highly recommended to manually load the table using the database console LOAD tool, as indicated in Week 2 Lab 1 Part II__. The only difference with that lab is that in Step 5 of the instructions you will need to click on create "(+) New Table" and specify the name of the table you want to create and then click "Next". 

##### Now open the Db2 console, open the LOAD tool, Select / Drag the .CSV file for the CHICAGO PUBLIC SCHOOLS dataset and load the dataset into a new table called __SCHOOLS__.

<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/uc4xjh1uxcc78ks1i18v668simioz4es.jpg"></a>

### Connect to the database
Let us now load the ipython-sql  extension and establish a connection with the database

In [15]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [16]:
# Enter the connection string for your Db2 on Cloud database instance below
# %sql ibm_db_sa://my-username:my-password@my-hostname:my-port/my-db-name/
%sql ibm_db_sa://hcg90289:bw10zlmgx7crb-gb@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB

'Connected: hcg90289@BLUDB'

### Query the database system catalog to retrieve table metadata

##### You can verify that the table creation was successful by retrieving the list of all tables in your schema and checking whether the SCHOOLS table was created

In [17]:
# type in your query to retrieve list of all tables in the database for your db2 schema (username)
%sql select TABSCHEMA, TABNAME, CREATE_TIME from SYSCAT.TABLES where TABSCHEMA='Db2-bo'
%sql select * from SYSCAT.TABLES where TABNAME = 'CENSUS'
# %sql select TABSCHEMA, TABNAME, CREATE_TIME from SYSCAT.TABLES \
#      where TABSCHEMA not in ('SYSIBM', 'SYSCAT', 'SYSSTAT', 'SYSIBMADM', 'SYSTOOLS', 'SYSPUBLIC')

 * ibm_db_sa://hcg90289:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.
 * ibm_db_sa://hcg90289:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


tabschema,tabname,owner,ownertype,TYPE,status,base_tabschema,base_tabname,rowtypeschema,rowtypename,create_time,alter_time,invalidate_time,stats_time,colcount,tableid,tbspaceid,card,npages,mpages,fpages,npartitions,nfiles,tablesize,overflow,tbspace,index_tbspace,long_tbspace,parents,children,selfrefs,keycolumns,keyindexid,keyunique,checkcount,datacapture,const_checked,pmap_id,partition_mode,log_attribute,pctfree,append_mode,REFRESH,refresh_time,LOCKSIZE,VOLATILE,row_format,property,statistics_profile,compression,rowcompmode,access_mode,clustered,active_blocks,droprule,maxfreespacesearch,avgcompressedrowsize,avgrowcompressionratio,avgrowsize,pctrowscompressed,logindexbuild,codepage,collationschema,collationname,collationschema_orderby,collationname_orderby,encoding_scheme,pctpagessaved,last_regen_time,secpolicyid,protectiongranularity,auditpolicyid,auditpolicyname,auditexceptionenabled,definer,oncommit,logged,onrollback,lastused,control,temporaltype,tableorg,extended_row_size,pctextendedrows,remarks
HCG90289,CENSUS,HCG90289,U,T,N,None,None,None,None,2018-12-01 18:40:11.703528,2018-12-01 18:40:11.703528,2018-12-01 18:40:11.703528,2018-12-01 18:44:20.340642,9,6,788,78,1,0,2,-1,-1,-1,0,hcg90289space1,None,None,0,0,0,0,0,0,0,N,YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY,1,,0,-1,N,,None,R,,N,,None,N,,F,None,0,N,999,0,0.0,57,0.0,None,1208,SYSIBM,IDENTITY,SYSIBM,IDENTITY,,0,2018-12-01 18:40:11.703528,0,,None,None,N,HCG90289,,,,2018-12-03,,N,R,N,-1.0,None


Double-click __here__ for a hint

<!--
In Db2 the system catalog table called SYSCAT.TABLES contains the table metadata
-->

Double-click __here__ for the solution.

<!-- Solution:

%sql select TABSCHEMA, TABNAME, CREATE_TIME from SYSCAT.TABLES where TABSCHEMA='YOUR-DB2-USERNAME'

or, you can retrieve list of all tables where the schema name is not one of the system created ones:

%sql select TABSCHEMA, TABNAME, CREATE_TIME from SYSCAT.TABLES \
      where TABSCHEMA not in ('SYSIBM', 'SYSCAT', 'SYSSTAT', 'SYSIBMADM', 'SYSTOOLS', 'SYSPUBLIC')
      
or, just query for a specifc table that you want to verify exists in the database
%sql select * from SYSCAT.TABLES where TABNAME = 'SCHOOLS'

-->

### Query the database system catalog to retrieve column metadata

##### The SCHOOLS table contains a large number of columns. How many columns does this table have?

In [4]:
# type in your query to retrieve the number of columns in the SCHOOLS table
%sql select count(*) from SYSCAT.COLUMNS where TABNAME = 'SCHOOLS'

 * ibm_db_sa://hcg90289:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


1
79


Double-click __here__ for a hint

<!--
In Db2 the system catalog table called SYSCAT.COLUMNS contains the column metadata
-->

Double-click __here__ for the solution.

<!-- Solution:

%sql select count(*) from SYSCAT.COLUMNS where TABNAME = 'SCHOOLS'

-->

Now retrieve the the list of columns in SCHOOLS table and their column type (datatype) and length.

In [23]:
# type in your query to retrieve all column names in the SCHOOLS table along with their datatypes and length
%sql select COLNAME, TYPENAME, LENGTH from SYSCAT.COLUMNS where TABNAME = 'SCHOOLS'
# %sql select distinct(NAME), COLTYPE, LENGTH from SYSIBM.SYSCOLUMNS where TBNAME = 'CENSUS'
# %sql select distinct(NAME), COLTYPE, LENGTH from SYSIBM.SYSCOLUMNS where TBNAME = 'CRIME'

 * ibm_db_sa://hcg90289:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


colname,typename,length
School_ID,INTEGER,4
Name_of_School,VARCHAR,65
"Elementary, Middle, or High School",VARCHAR,2
Street_Address,VARCHAR,30
City,VARCHAR,7
State,VARCHAR,2
ZIP_Code,INTEGER,4
Phone_Number,VARCHAR,14
Link,VARCHAR,78
Network_Manager,VARCHAR,40


Double-click __here__ for the solution.

<!-- Solution:

%sql select COLNAME, TYPENAME, LENGTH from SYSCAT.COLUMNS where TABNAME = 'SCHOOLS'

or

%sql select distinct(NAME), COLTYPE, LENGTH from SYSIBM.SYSCOLUMNS where TBNAME = 'SCHOOLS'

-->

### Questions
1. Is the column name for the "SCHOOL ID" attribute in upper or mixed case?
1. What is the "Community Area Name" field called in your table? Have the spaces " " between the words been replaced by some other character?
1. Have the paranthesis (round brackets) in the "College Enrollment (number of students)" attribute been replaced by some other character? 
1. Are there any columns in whose names the spaces and paranthesis (round brackets) have not been replaced by the underscore character "_"?

## Problems

### Problem 1

##### How many Elementary Schools are in the dataset?

In [22]:
%sql select count(*) from CRIME where "Location_Description" = 'GAS STATION'

 * ibm_db_sa://hcg90289:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


1
6


Double-click __here__ for a hint

<!--
Which column specifies the school type e.g. 'ES', 'MS', 'HS'?
-->

Double-click __here__ for another hint

<!--
Does the column name have mixed case, spaces or other special characters?
If so, ensure you use double quotes around the "Name of the Column"
-->

Double-click __here__ for the solution.

<!-- Solution:

%sql select count(*) from SCHOOLS where "Elementary, Middle, or High School" = 'ES'

Correct answer: 462

-->

### Problem 2

##### What is the highest Safety Score?

In [24]:
%sql select MAX("Safety_Score") AS MAX_SAFETY_SCORE from SCHOOLS

 * ibm_db_sa://hcg90289:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


max_safety_score
99


Double-click __here__ for a hint

<!--
Use the MAX() function
-->

Double-click __here__ for the solution.

<!-- Hint:

%sql select MAX("Safety_Score") AS MAX_SAFETY_SCORE from SCHOOLS
Correct answer: 99
-->


### Problem 3

##### Which schools have highest Safety Score?

In [15]:
%sql select "Name_of_School", "Safety_Score" from SCHOOLS where \
  "Safety_Score"= (select MAX("Safety_Score") from SCHOOLS)

 * ibm_db_sa://hcg90289:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


Name_of_School,Safety_Score
Abraham Lincoln Elementary School,99
Alexander Graham Bell Elementary School,99
Annie Keller Elementary Gifted Magnet School,99
Augustus H Burley Elementary School,99
Edgar Allan Poe Elementary Classical School,99
Edgebrook Elementary School,99
Ellen Mitchell Elementary School,99
James E McDade Elementary Classical School,99
James G Blaine Elementary School,99
LaSalle Elementary Language Academy,99


Double-click __here__ for the solution.

<!-- Solution:
In the previous problem we found out that the highest Safety Score is 99, so we can use that as an input in the where clause:

%sql select "Name_of_School", "Safety_Score" from SCHOOLS where "Safety_Score" = 99

or, a better way:

%sql select "Name_of_School", "Safety_Score" from SCHOOLS where \
  "Safety_Score"= (select MAX("Safety_Score") from SCHOOLS)

Correct answer: several schools with with Safety Score of 99.
-->


### Problem 4

##### What are the top 10 schools with the highest "Average Student Attendance"?


In [25]:
%sql select "Community_Area_Name", "College_Enrollment__number_of_students_" from SCHOOLS \
    order by "Community_Area_Name" 

 * ibm_db_sa://hcg90289:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


Community_Area_Name,College_Enrollment__number_of_students_
ALBANY PARK,266
ALBANY PARK,1023
ALBANY PARK,1299
ALBANY PARK,919
ALBANY PARK,359
ALBANY PARK,1490
ALBANY PARK,269
ALBANY PARK,1239
ARCHER HEIGHTS,3320
ARCHER HEIGHTS,1503


Double-click __here__ for the solution.

<!-- Solution:

%sql select "Name_of_School", "Average_Student_Attendance" from SCHOOLS \
    order by "Average_Student_Attendance" desc nulls last limit 10 

-->

### Problem 5

##### Retrieve the list of 5 Schools with the lowest Average Student Attendance sorted in ascending order based on attendance

In [18]:
%sql SELECT "Name_of_School", "Average_Student_Attendance"  \
     from SCHOOLS \
     order by "Average_Student_Attendance" \
     fetch first 5 rows only

 * ibm_db_sa://hcg90289:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


Name_of_School,Average_Student_Attendance
Richard T Crane Technical Preparatory High School,57.9
Barbara Vick Early Childhood & Family Center,60.9
Dyett High School,62.5
Wendell Phillips Academy High School,63.0
Orr Academy High School,66.3


Double-click __here__ for the solution.

<!-- Solution:

%sql SELECT "Name_of_School", "Average_Student_Attendance"  \
     from SCHOOLS \
     order by "Average_Student_Attendance" \
     fetch first 5 rows only

-->


### Problem 6

##### Now remove the '%' sign from the above result set for Average Student Attendance column

In [19]:
%sql SELECT "Name_of_School", REPLACE("Average_Student_Attendance", '%', '') \
     from SCHOOLS \
     order by "Average_Student_Attendance" \
     fetch first 5 rows only

 * ibm_db_sa://hcg90289:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


Name_of_School,2
Richard T Crane Technical Preparatory High School,57.9
Barbara Vick Early Childhood & Family Center,60.9
Dyett High School,62.5
Wendell Phillips Academy High School,63.0
Orr Academy High School,66.3


Double-click __here__ for a hint

<!--
Use the REPLACE() function to replace '%' with ''
See documentation for this function at:
https://www.ibm.com/support/knowledgecenter/en/SSEPGG_10.5.0/com.ibm.db2.luw.sql.ref.doc/doc/r0000843.html
-->

Double-click __here__ for the solution.

<!-- Hint:

%sql SELECT "Name_of_School", REPLACE("Average_Student_Attendance", '%', '') \
     from SCHOOLS \
     order by "Average_Student_Attendance" \
     fetch first 5 rows only

-->


### Problem 7

##### Which Schools have Average Student Attendance lower than 70%?

In [12]:
%sql SELECT "Name_of_School", "Average_Student_Attendance"  \
     from SCHOOLS \
     where CAST ( REPLACE("Average_Student_Attendance", '%', '') AS DOUBLE ) < 70 \
     order by "Average_Student_Attendance"

 * ibm_db_sa://hcg90289:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


Name_of_School,Average_Student_Attendance
Richard T Crane Technical Preparatory High School,57.9
Barbara Vick Early Childhood & Family Center,60.9
Dyett High School,62.5
Wendell Phillips Academy High School,63.0
Orr Academy High School,66.3
Manley Career Academy High School,66.8
Chicago Vocational Career Academy High School,68.8
Roberto Clemente Community Academy High School,69.6


Double-click __here__ for a hint

<!--
The datatype of the "Average_Student_Attendance" column is varchar.
So you cannot use it as is in the where clause for a numeric comparison.
First use the CAST() function to cast it as a DECIMAL or DOUBLE
e.g. CAST("Column_Name" as DOUBLE)
or simply: DECIMAL("Column_Name")
-->

Double-click __here__ for another hint

<!--
Don't forget the '%' age sign needs to be removed before casting
-->

Double-click __here__ for the solution.

<!-- Solution:

%sql SELECT "Name_of_School", "Average_Student_Attendance"  \
     from SCHOOLS \
     where CAST ( REPLACE("Average_Student_Attendance", '%', '') AS DOUBLE ) < 70 \
     order by "Average_Student_Attendance"
     
or,

%sql SELECT "Name_of_School", "Average_Student_Attendance"  \
     from SCHOOLS \
     where DECIMAL ( REPLACE("Average_Student_Attendance", '%', '') ) < 70 \
     order by "Average_Student_Attendance"

-->


### Problem 8

##### Get the total __College Enrollment (number of students)__ for each Community Area

In [21]:
%sql select "Community_Area_Name", avg("College_Enrollment__number_of_students_") AS TOTAL_ENROLLMENT \
   from SCHOOLS \
   group by "Community_Area_Name" \
   order by TOTAL_ENROLLMENT desc \
   fetch first 10 rows only

 * ibm_db_sa://hcg90289:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


Community_Area_Name,total_enrollment
ARCHER HEIGHTS,2411.500000
MONTCLARE,1317.000000
WEST ELSDON,1233.333333
BRIGHTON PARK,1205.875000
BELMONT CRAGIN,1198.833333
NORTH CENTER,1077.285714
EAST SIDE,1061.000000
WEST LAWN,1051.750000
CHICAGO LAWN,1012.285714
HERMOSA,993.750000


Double-click __here__ for a hint

<!--
Verify the exact name of the Enrollment column in the database
Use the SUM() function to add up the Enrollments for each Community Area
-->

Double-click __here__ for another hint

<!--
Don't forget to group by the Community Area
-->

Double-click __here__ for the solution.

<!-- Solution:

%sql select "Community_Area_Name", sum("College_Enrollment__number_of_students_") AS TOTAL_ENROLLMENT \
   from SCHOOLS \
   group by "Community_Area_Name" 

-->


### Problem 9

##### Get the 5 Community Areas with the least total __College Enrollment (number of students)__  sorted in ascending order 

In [28]:
%sql select sum("Elementary, Middle, or High School") AS Most_No_ of_Schools from SCHOOLS \
order by TOTAL_ENROLLMENT desc \
fetch first 10 rows only

 * ibm_db_sa://hcg90289:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
(ibm_db_dbi.ProgrammingError) ibm_db_dbi::ProgrammingError: SQLNumResultCols failed: [IBM][CLI Driver][DB2/LINUXX8664] SQL0104N  An unexpected token "AS" was found following "le, or High School")".  Expected tokens may include:  "AND".  SQLSTATE=42601 SQLCODE=-104 [SQL: 'select sum("Elementary, Middle, or High School") AS Most_No_ of_Schools from SCHOOLS order by TOTAL_ENROLLMENT desc fetch first 10 rows only'] (Background on this error at: http://sqlalche.me/e/f405)


Double-click __here__ for a hint

<!--
Order the previous query and limit the number of rows you fetch
-->

Double-click __here__ for the solution.

<!-- Solution:

%sql select "Community_Area_Name", sum("College_Enrollment__number_of_students_") AS TOTAL_ENROLLMENT \
   from SCHOOLS \
   group by "Community_Area_Name" \
   order by TOTAL_ENROLLMENT asc \
   fetch first 5 rows only

-->


## Summary

##### In this lab you learned how to work with a real word dataset using SQL and Python. You learned how to query columns with spaces or special characters in their names and with mixed names. You also used built in database functions.

Copyright &copy; 2018 [cognitiveclass.ai](cognitiveclass.ai?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).
